In [9]:
!pip install rouge

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
%cd code2seq

/home/ec2-user/SageMaker/code2seq/code2seq


In [4]:
!ls

baseline_tokenization  hs_err_pid19158.log  hs_err_pid29043.log
code2seq.py	       hs_err_pid19189.log  hs_err_pid29126.log
common.py	       hs_err_pid19314.log  hs_err_pid29214.log
config.py	       hs_err_pid19664.log  hs_err_pid29253.log
CSharpExtractor        hs_err_pid19668.log  hs_err_pid29289.log
data		       hs_err_pid19669.log  hs_err_pid29571.log
error_log.txt	       hs_err_pid19708.log  hs_err_pid29678.log
extractor.py	       hs_err_pid19846.log  hs_err_pid29771.log
hs_err_pid10262.log    hs_err_pid20044.log  hs_err_pid30214.log
hs_err_pid11277.log    hs_err_pid20181.log  hs_err_pid30243.log
hs_err_pid11398.log    hs_err_pid20230.log  hs_err_pid30281.log
hs_err_pid11535.log    hs_err_pid20361.log  hs_err_pid30551.log
hs_err_pid11987.log    hs_err_pid20415.log  hs_err_pid30571.log
hs_err_pid12045.log    hs_err_pid20529.log  hs_err_pid30823.log
hs_err_pid12383.log    hs_err_pid20569.log  hs_err_pid30972.log
hs_err_pid12522.log    hs_err_pid20663.log  hs_err_pid30997.log
hs_err_

In [ ]:
!bash preprocess.sh

In [3]:
ls

baseline_tokenization/  hs_err_pid24388.log
code2seq.py             hs_err_pid24465.log
common.py               hs_err_pid24480.log
config.py               hs_err_pid24533.log
CSharpExtractor/        hs_err_pid24570.log
data/                   hs_err_pid24572.log
error_log.txt           hs_err_pid24588.log
extractor.py            hs_err_pid2459.log
hs_err_pid10262.log     hs_err_pid24616.log
hs_err_pid10319.log     hs_err_pid24640.log
hs_err_pid10434.log     hs_err_pid24675.log
hs_err_pid10732.log     hs_err_pid24680.log
hs_err_pid1117.log      hs_err_pid24793.log
hs_err_pid11181.log     hs_err_pid2480.log
hs_err_pid11277.log     hs_err_pid24837.log
hs_err_pid11353.log     hs_err_pid24896.log
hs_err_pid11398.log     hs_err_pid24904.log
hs_err_pid1141.log      hs_err_pid24905.log
hs_err_pid11535.log     hs_err_pid24907.log
hs_err_pid11550.log     hs_err_pid24927.log
hs_err_pid11637.log     hs_err_pid24985.log
hs_err_pid11920.log     hs_err_pid25052.log
hs_err_pid1195.log      hs_err_pid

In [4]:
#first reproduced the result with preprocessed data
#no link from preprocessed data to original project 
#need to make a map by processing again by myself
# huge time for compuation and the code is also convoluted
import pickle
import numpy as np
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
    
def process_file(file_path, data_file_role='test', dataset_name="own_test", max_contexts=200, max_data_contexts=1000):
    sum_total = 0
    sum_sampled = 0
    project_loc = {}
    total = 0
    max_unfiltered = 0
    max_contexts_to_sample = max_data_contexts if data_file_role == 'test' else max_contexts
    output_path = '{}.{}.c2s'.format(dataset_name, data_file_role)
    with open(output_path, 'w') as outfile:
        with open(file_path, 'r') as file:
            for line in file:
                if len(line.strip())==0:
                    continue
                if '##dir' in line:
                    project_loc[total] = line.rstrip('\n').replace("####dir:","").split('/')[3]
                    continue
                parts = line.rstrip('\n').split(' ')
                target_name = parts[0]
                contexts = parts[1:]

                if len(contexts) > max_unfiltered:
                    max_unfiltered = len(contexts)

                sum_total += len(contexts)
                if len(contexts) > max_contexts_to_sample:
                    contexts = np.random.choice(contexts, max_contexts_to_sample, replace=False)

                sum_sampled += len(contexts)

                csv_padding = " " * (max_data_contexts - len(contexts))
                total += 1
                outfile.write(target_name + ' ' + " ".join(contexts) + csv_padding + '\n')
    save_obj(project_loc,'project_loc')
    print('File: ' + file_path)
    print('Average total contexts: ' + str(float(sum_total) / total))
    print('Average final (after sampling) contexts: ' + str(float(sum_sampled) / total))
    print('Total examples: ' + str(total))
    print('Max number of contexts per word: ' + str(max_unfiltered))
    return total


In [5]:
process_file('own_test.txt')

File: own_test.txt
Average total contexts: 183.95688537156352
Average final (after sampling) contexts: 152.7865462304999
Total examples: 482690
Max number of contexts per word: 180509


482690

In [9]:
with open('code2seq/own_test.txt', 'r') as file:
    for line in file:
        if '##dir' in line:
            print("-------------")
            print(line)
            print("-------------")

-------------
##dir:raw_data/java-med/test/Trinea__android-auto-scroll-view-pager

-------------
-------------
##dir:raw_data/java-med/test/konifar__android-material-design-icon-generator-plugin

-------------
-------------
##dir:raw_data/java-med/test/yarolegovich__DiscreteScrollView

-------------
-------------
##dir:raw_data/java-med/test/kanytu__android-parallax-recyclerview

-------------
-------------
##dir:raw_data/java-med/test/daimajia__NumberProgressBar

-------------
-------------
handle|failed|request void,Void0|Mth|Nm1,METHOD_NAME void,Void0|Mth|Prm|VDID0,jrt|req void,Void0|Mth|Prm|Cls1,jrt|client|config|request METHOD_NAME,Nm1|Mth|Prm|VDID0,jrt|req METHOD_NAME,Nm1|Mth|Prm|Cls1,jrt|client|config|request METHOD_NAME,Nm1|Mth|Prm|VDID0,sub METHOD_NAME,Nm1|Mth|Prm|Cls|Cls0,config|instance jrt|req,VDID0|Prm|Cls1,jrt|client|config|request jrt|req,VDID0|Prm|Mth|Prm|VDID0,sub jrt|req,VDID0|Prm|Mth|Prm|Cls|Cls0,config|instance jrt|req,VDID0|Prm|Mth|Prm|VDID0,connection jrt|req,VDID

In [14]:
!tail -40 code2seq/own_test.txt

features|helper block,Cls0|Mth|Nm1,METHOD_NAME block,Cls0|Mth|Prm|VDID0,features block,Cls0|Mth|Prm|Prim1,double METHOD_NAME,Nm1|Mth|Prm|VDID0,features METHOD_NAME,Nm1|Mth|Prm|Prim1,double METHOD_NAME,Nm1|Mth|Bk|If|Cal0|Nm0,page|builder METHOD_NAME,Nm1|Mth|Bk|If|Cal0|Nm2,is|full METHOD_NAME,Nm1|Mth|Bk|If|Bk|Ex|Cal0|Nm0,page|builder METHOD_NAME,Nm1|Mth|Bk|If|Bk|Ex|Cal0|Nm2,reset METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|Cls0,block|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|VDID0,map|block|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|Cal1|Nm0,page|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|Cal1|IntEx2,0 METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|Cal1|Nm3,get|block|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|Cls0,block|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|VDID0,block|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|Cal1|Nm0,map|block|builder METHOD_NAME,Nm1|Mth|Bk|Ex|VDE|VD|Cal1|Nm2,begin|block|entry METHOD_NAME,Nm1|Mth|Bk|For|Ls|Nm0,i METHOD_NAME,Nm1|Mth|Bk|For|Ls|Fld1|Nm0,features METHOD_NAME,Nm1|Mth|Bk|For|Ls|Fld1|Nm2,length

In [42]:
ls

baseline_tokenization/  hs_err_pid19007.log  hs_err_pid28671.log
code2seq.py             hs_err_pid19158.log  hs_err_pid28698.log
common.py               hs_err_pid19189.log  hs_err_pid29043.log
config.py               hs_err_pid19314.log  hs_err_pid29126.log
CSharpExtractor/        hs_err_pid19664.log  hs_err_pid29214.log
data/                   hs_err_pid19668.log  hs_err_pid29253.log
error_log.txt           hs_err_pid19669.log  hs_err_pid29289.log
extractor.py            hs_err_pid19708.log  hs_err_pid29571.log
hs_err_pid10262.log     hs_err_pid19846.log  hs_err_pid29678.log
hs_err_pid11277.log     hs_err_pid20044.log  hs_err_pid29771.log
hs_err_pid11398.log     hs_err_pid20181.log  hs_err_pid30214.log
hs_err_pid11535.log     hs_err_pid20230.log  hs_err_pid30243.log
hs_err_pid11987.log     hs_err_pid20361.log  hs_err_pid30281.log
hs_err_pid12045.log     hs_err_pid20415.log  hs_err_pid30551.log
hs_err_pid12383.log     hs_err_pid20529.log  hs_err_pid30571.log
hs_err_pid12522.log     h

In [26]:
!head -10 code2seq/own_test.txt

get|display|name string,Cls0|Mth|Nm1,METHOD_NAME string,Cls0|Mth|Bk|Ret|Nm0,display|name METHOD_NAME,Nm1|Mth|Bk|Ret|Nm0,display|name
get|resource int,Prim0|Mth|Nm1,METHOD_NAME int,Prim0|Mth|Bk|Ret|Nm0,res METHOD_NAME,Nm1|Mth|Bk|Ret|Nm0,res
set|scroll|duration|factor void,Void0|Mth|Nm1,METHOD_NAME void,Void0|Mth|Prm|VDID0,scroll|factor void,Void0|Mth|Prm|Prim1,double METHOD_NAME,Nm1|Mth|Prm|VDID0,scroll|factor METHOD_NAME,Nm1|Mth|Prm|Prim1,double METHOD_NAME,Nm1|Mth|Bk|Ex|As0|Fld0|This0,this METHOD_NAME,Nm1|Mth|Bk|Ex|As0|Fld0|Nm2,scroll|factor METHOD_NAME,Nm1|Mth|Bk|Ex|As0|Nm1,scroll|factor scroll|factor,VDID0|Prm|Prim1,double scroll|factor,VDID0|Prm|Mth|Bk|Ex|As0|Fld0|This0,this scroll|factor,VDID0|Prm|Mth|Bk|Ex|As0|Fld0|Nm2,scroll|factor scroll|factor,VDID0|Prm|Mth|Bk|Ex|As0|Nm1,scroll|factor double,Prim1|Prm|Mth|Bk|Ex|As0|Fld0|This0,this double,Prim1|Prm|Mth|Bk|Ex|As0|Fld0|Nm2,scroll|factor double,Prim1|Prm|Mth|Bk|Ex|As0|Nm1,scroll|factor this,This0|Fld0|Nm2,scroll|factor this,This0|

In [2]:
!wget https://s3.amazonaws.com/code2seq/model/java-large/java-large-model.tar.gz
!tar -xvzf java-large-model.tar.gz

--2020-12-03 11:06:41--  https://s3.amazonaws.com/code2seq/model/java-large/java-large-model.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.230.45
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.230.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143205968 (137M) [application/x-tar]
Saving to: ‘java-large-model.tar.gz’

java-large-model.ta 100%[===================>] 136.57M  53.6MB/s    in 2.5s    

2020-12-03 11:06:43 (53.6 MB/s) - ‘java-large-model.tar.gz’ saved [143205968/143205968]

models/java-large-model/model_iter52.release.data-00000-of-00001
models/java-large-model/model_iter52.release.index
models/java-large-model/model_iter52.release.dict


In [45]:
!python3 code2seq.py --load models/java-large-model/model_iter52.release --test own_test.test.c2s




Loading dictionaries from: models/java-large-model/model_iter52.release
Done loading dictionaries
Created model
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
delimiter is deprecated, please use sep instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
Instructions for updati

In [5]:
%mkdir data
%cd data
!wget https://s3.amazonaws.com/code2seq/datasets/java-med-preprocessed.tar.gz
!tar -xvzf java-med-preprocessed.tar.gz

mkdir: cannot create directory ‘data’: File exists
/home/ec2-user/SageMaker/code2seq/code2seq/data
--2020-12-03 15:40:13--  https://s3.amazonaws.com/code2seq/datasets/java-med-preprocessed.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.132.101
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.132.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2391834044 (2.2G) [application/x-tar]
Saving to: ‘java-med-preprocessed.tar.gz’

java-med-preprocess 100%[===================>]   2.23G  49.1MB/s    in 42s     

2020-12-03 15:40:55 (54.1 MB/s) - ‘java-med-preprocessed.tar.gz’ saved [2391834044/2391834044]

java-med/java-med.dict.c2s
java-med/java-med.test.c2s
java-med/java-med.train.c2s
java-med/java-med.val.c2s


In [14]:
!python3 code2seq.py --load models/java-large-model/model_iter52.release --test data/java-med/java-med.test.c2s




Loading dictionaries from: models/java-large-model/model_iter52.release
Done loading dictionaries
Created model
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
delimiter is deprecated, please use sep instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
Instructions for updati

In [10]:
#After changing with my own test data
!python3 code2seq.py --load models/java-large-model/model_iter52.release --test data/java-med/java-med.test.c2s




Loading dictionaries from: models/java-large-model/model_iter52.release
Done loading dictionaries
Created model
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
delimiter is deprecated, please use sep instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
Instructions for updati

In [15]:
ls data/java-med/

java-med.dict.c2s  java-med.test.c2s  java-med.train.c2s  java-med.val.c2s


In [18]:
!wc -l data/java-med/java-med.test.c2s

419914 data/java-med/java-med.test.c2s


In [7]:
#My own data
!wc -l data/java-med/java-med.test.c2s

482690 data/java-med/java-med.test.c2s
